In [ ]:
import wradlib as wrl
import warnings
warnings.filterwarnings('ignore')
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()
import numpy as np

In [ ]:
import os
import glob
import xarray as xr
import datetime as dt
import dateutil.parser as dparser
import matplotlib.pyplot as pl

In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
import hvplot
import hvplot.xarray
import hvplot.pandas

# Timeseries/QVP ODIM_H5 using XARRAY

This shows loading of a series of sweeps of BoXPol Radar using `wradlib.io.xarray.OdimH5`

In [ ]:
from wradlib.io.xarray import OdimH5

## Function Definitons

### Check coordinates

The CfRadial2.0 standard has one `time`/`azimuth` dimension/coordinate already defined. We need to take some precautions to check this for problems and prevent further code from breaking.

In [ ]:
def check_azimuth(ds):
    """reindex azimuth - missing rays, double rays
    """
    dim = ds.azimuth.dims[0]
    res = ds.azimuth.diff(dim).median().round(decimals=1)
    azr = np.arange(res/2., 360, res)
    ds = ds.sortby('azimuth').reindex(azimuth=azr, method='nearest', tolerance=res/4.)
    return ds

def check_elevation(ds):
    """assign elevation - might just use elevation from root-ds or median
    """
    elr = np.ones(ds.azimuth.shape) * np.round(np.nanmedian(ds.elevation.values), decimals=1)
    ds = ds.assign_coords(elevation=(['azimuth'], elr))
    return ds

def check_time(ds):
    # rename time coordinate/variable and add new time
    ds = ds.rename({'time': 'rtime'})#.expand_dims('time')
    ds = ds.assign({'start_time': (['time'], [ds['rtime'].values[0]])})
    # assign new time coordinate
    ds = ds.assign({'time': (['time'], ds['start_time'])})
    return ds

#### Georeference Dataset

In [ ]:
def georeference_dataset(ds, is_ppi):
    """Georeference Dataset.

    This function adds georeference data to xarray dataset `ds`.

    Parameters
    ----------
    ds : xarray dataset
    is_ppi : bool
        PPI/RHI flag
    """
    # adding xyz aeqd-coordinates
    site = (ds.coords['longitude'].values, ds.coords['latitude'].values,
            ds.coords['altitude'].values)
    dim0 = ds['azimuth'].dims[0]
    xyz, aeqd = wrl.georef.spherical_to_xyz(ds['range'],
                                            ds['azimuth'],
                                            ds['elevation'],
                                            site,
                                            squeeze=True)
    gr = np.sqrt(xyz[..., 0] ** 2 + xyz[..., 1] ** 2)
    ds.coords['x'] = ([dim0, 'range'], xyz[..., 0])
    ds.coords['y'] = ([dim0, 'range'], xyz[..., 1])
    ds.coords['z'] = ([dim0, 'range'], xyz[..., 2])
    ds.coords['gr'] = ([dim0, 'range'], gr)

    # adding rays, bins coordinates
    if is_ppi:
        bins, rays = np.meshgrid(ds['range'],
                                 ds['azimuth'],
                                 indexing='xy')
    else:
        bins, rays = np.meshgrid(ds['range'],
                                 ds['elevation'],
                                 indexing='xy')
    ds.coords['rays'] = ([dim0, 'range'], rays)
    ds.coords['bins'] = ([dim0, 'range'], bins)
    
    return ds

#### Utility Functions

In [ ]:
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
        
def get_data_path(inpath):
    """ Get data path (automount) 
    """
    return os.path.join(inpath, '{year}/{year}-{month:02d}/'
                                '{year}-{month:02d}-{day:02d}/{scan}')

def get_file_name_new():
    return ('{scan}_12345_{year}{month:02d}{day:02d}{hour:02d}{mintens}'
            '[{minones0}-{minones1}]*')

def get_file_name_old():
    return ('{year}-{month:02d}-{day:02d}--{hour:02d}:{mintens}'
            '[{minones0}-{minones1}]*')

def import_dates(date_string):
    return dparser.parse(date_string)

def get_time_offset(val):
    t0 = dt.datetime.utcfromtimestamp(val.astype(int) * 1e-9)
    t1 = dt.datetime(t0.year, t0.month, t0.day, t0.hour, t0.minute)
    return t0-t1

In [ ]:
def create_timeseries(flist, **kwargs):
    disable = len(flist) == 1
    root = []
    sweep = []
    first = True
    i = 0
    for fname in tqdm(flist, 'scans', ascii=True, disable=disable):
        # Handle missing scans
        try:
            fname = fname[0]
        except IndexError:
            continue
        ds0 = OdimH5(fname, flavour='GAMIC', **kwargs)
        root.append(ds0['root'])
        sweep.append(ds0['sweep_1'].pipe(check_azimuth).pipe(check_elevation).pipe(check_time))

    ds_root = xr.concat(root, dim='time', data_vars='different')
    ds_sweep = xr.concat(sweep, dim='time', data_vars='different', coords='different')
    return {'root': ds_root, 'sweep_1': ds_sweep}

## Load timeseries of sweeps

First, we need to claim all files which corresponds to the specific sweep and time range. So we take a wanted `start_time`, `stop_time` and `scan` and retrieve all fitting ppi sweeps.

In [ ]:
# recent boxpol event
start_time = dt.datetime(2019, 5, 21, 14)
stop_time = dt.datetime(2019, 5, 21, 16)

inpath = '/automount/radar/scans'
#inpath = '/automount/radar-archiv/scans'

scan = 'n_ppi_180deg'
startmin = 0
stopmin = 3

drange = [result for result in
          perdelta(start_time, stop_time, dt.timedelta(minutes=5))]

path = get_data_path(inpath)
path = os.path.join(path, get_file_name_new())
path

### create file list

To retrieve the sweeps for every timestep `glob` module is used with the prepared path variable. Finally, one timestep is removed from the list to simulate a missing sweep.

In [ ]:
flist = [glob.glob(path.format(year=t.year, month=t.month, day=t.day, 
                               scan=scan,  
                               hour=t.hour, mintens=int(t.minute / 10),
                               minones0=t.minute % 10 + startmin,
                               minones1=t.minute % 10 + stopmin)) for t in drange]
print(len(flist))
flist[10] = []
flist[11]

### load timeseries

Here `create_timeseries` is used with the given keyword arguments `dim0='azimuth'` and `georef=False`. This will use `azimuth` as first dimension (instead of `time`) and will not add georeferenced coordinates to everey sweep.

In [ ]:
ts = create_timeseries(flist, dim0='azimuth', georef=False)

In [ ]:
list(ts.keys())

In [ ]:
ts['root']

In [ ]:
ts['sweep_1']

## reindex time - fix missing timesteps

For a timeseries of sweeps not all measurements might be available. So we have to deal with missing sweeps at any position in the series. The solution is to reindex along the `time`-dimension using the precomputed `drange` array (5 minute resolution).

First we transform the `drange` data type to `np.datetime64` for use with xarray. Then we reindex along the time dimension using nearest neighbour with a tolerance of 150 s (2.5 minutes).

Note that the resulting `time`-coordinate is in a fixed 5 minute interval (00:00, 00:05, ..., 00:55, 01:00, 01:05, ...).

In [ ]:
ntime = np.array([np.datetime64(dr) for dr in drange])
ts['sweep_1'] = ts['sweep_1'].reindex(time=ntime, method='nearest', tolerance=150e+9)
ts['sweep_1']

## Georeference timeseries

Georeferenced AEQD coordinates xyz, gr and rays, bins can be attached using `georeference_dataset` function.

In [ ]:
ts['sweep_1'] = ts['sweep_1'].pipe(georeference_dataset, is_ppi=True)
ts['sweep_1']

## Assign height-coord from z-coord

For QVP a height-coordinate is needed. It is derived from the dataset's `z`-coordinate.

In [ ]:
ts['sweep_1'] = ts['sweep_1'].assign_coords(height=ts['sweep_1'].z.mean('azimuth'))
ts['sweep_1']

## Inspect the data using `hvplot`

In [ ]:
dbz_plot = ts['sweep_1'].hvplot.quadmesh(groupby='time', 
                                          x='x', y='y', 
                                          z='DBZH', 
                                          rasterize=True, 
                                          clim=(0,50), cmap='Spectral',
                                          width=600, height=500)
dbz_plot

## Create Simple QVP

The QVP is created using a median along the `azimuth` dimension. This will collapse the `azimuth` dimension and we obtain an output dataset of `time`x`range`. Note that the `azimuth`-dimension is gets removed and all variables which cannot be collapsed correctly (xyz-coordinates etc.) will be removed.

Generally all available numpy statistic functions can be used here (eg. 'mean', 'std', etc.)

In [ ]:
qvp = ts['sweep_1'].median('azimuth')

In [ ]:
qvp

### Simple QVP Plot

In [ ]:
fig, axes = pl.subplots(ncols=2, nrows=2, figsize=(16,16))
print(axes)
qvp.DBZH.where(qvp.DBZH>0).plot(x='time', y='height', ax=axes[0,0], vmin=0, vmax=50)
qvp.RHOHV.where(qvp.DBZH>0).plot(x='time', y='height', ax=axes[0,1], vmin=0, vmax=1)
qvp.ZDR.where(qvp.DBZH>0).plot(x='time', y='height', ax=axes[1,0], vmin=-2, vmax=4)
qvp.PHIDP.where(qvp.DBZH>0).plot(x='time', y='height', ax=axes[1,1], vmin=75, vmax=95)
pl.tight_layout()

[ax.set_ylim(0, 7000) for ax in axes.flat]